In [2]:
from mxnet import gluon
from mxnet import autograd
import mxnet as mx

In [62]:
class MLR(gluon.nn.Block):
    
    def __init__(self, k, x_dim, **kwargs):
        super(MLR, self).__init__(**kwargs)
        self.k = k
        with self.name_scope():
            self.u = self.params.get("weight_u", shape=(x_dim, self.k))
            self.u_b = self.params.get("bias_u", shape=(self.k,))
            self.w = self.params.get("weight_w", shape=(x_dim, self.k))
            self.w_b = self.params.get("bias_w", shape=(self.k,))
    
    def forward(self, x):
        """
        mlr有三个部分, 第一部分的分子, 第一部分的分母, 第二部分
        现在分别实现, pai_son, pai_mo, lr_part
        :type x: mx.nd.array
        :param x: 
        :return: 
        """
        # pai的部分
        pai_son = mx.nd.exp(mx.nd.dot(X, self.u.data()) + self.u_b.data())
        pai_mo = mx.nd.sum(pai_son, axis=1)
        pai = mx.nd.broadcast_div(pai_son, pai_mo.reshape((-1, 1)))
        
        # lr部分
        lr_part_wx = mx.nd.dot(X, self.w.data()) + self.w_b.data()
        lr_part = 1 / (1 + mx.nd.exp(-lr_part_wx))
        
        # 结果相乘求和
        result = pai * lr_part
        return mx.nd.sum(result, axis=1)

In [49]:
test_u = mx.nd.array([[1, 2, 3], [4, 5, 6]])

In [50]:
test_b = mx.nd.array([1, 2])

In [65]:
X = mx.nd.array([[1, 1, 1], [2, 2, 2], [3, 3, 3], [1, 2, 3]])
X


[[ 1.  1.  1.]
 [ 2.  2.  2.]
 [ 3.  3.  3.]
 [ 1.  2.  3.]]
<NDArray 4x3 @cpu(0)>

In [23]:
temnp1 = mx.nd.dot(X, test_u.transpose())
temnp1


[[  6.  15.]
 [ 12.  30.]
 [ 18.  45.]]
<NDArray 3x2 @cpu(0)>

In [33]:
temnp1 + test_b


[[  7.  17.]
 [ 13.  32.]
 [ 19.  47.]]
<NDArray 3x2 @cpu(0)>

In [32]:
temp1sum = mx.nd.sum(temnp1, axis=1)
temp1sum


[ 21.  42.  63.]
<NDArray 3 @cpu(0)>

In [45]:
zzz = mx.nd.broadcast_div(temnp1, temp1sum.reshape((3, 1)))

In [34]:
test_w = mx.nd.array([[1, 2, 1], [4, 2, 1]])

In [37]:
temnp2 = mx.nd.dot(X, test_w.transpose())
temnp2


[[  4.   7.]
 [  8.  14.]
 [ 12.  21.]]
<NDArray 3x2 @cpu(0)>

In [40]:
temnp3 = mx.nd.exp(-temnp2)
temnp3


[[  1.83156393e-02   9.11881973e-04]
 [  3.35462624e-04   8.31528723e-07]
 [  6.14421242e-06   7.58256069e-10]]
<NDArray 3x2 @cpu(0)>

In [44]:
zz = 1 / (1 + temnp3)

In [46]:
zz * zzz


[[ 0.28057536  0.71363503]
 [ 0.28561848  0.71428514]
 [ 0.28571254  0.71428573]]
<NDArray 3x2 @cpu(0)>

In [63]:
mlr = MLR(2, 3)
mlr.initialize()

In [66]:
mlr(X)


[ 0.47932017  0.46543142  0.45144606  0.44385868]
<NDArray 4 @cpu(0)>

In [67]:
xs = mx.symbol.Variable("a")

In [78]:
X[:, 0]


[ 1.  2.  3.  1.]
<NDArray 4 @cpu(0)>

In [74]:
mx.nd.take(X, [[0,1],[1,2]])

AssertionError: Argument indices must have NDArray type, but got [[0, 1], [1, 2]]

In [79]:
a = mx.nd.slice_axis(X, axis=1, begin=0, end=1)

In [80]:
b = mx.nd.slice_axis(X, axis=1, begin=1, end=2)

In [81]:
a


[[ 1.]
 [ 2.]
 [ 3.]
 [ 1.]]
<NDArray 4x1 @cpu(0)>

In [82]:
b


[[ 1.]
 [ 2.]
 [ 3.]
 [ 2.]]
<NDArray 4x1 @cpu(0)>

In [84]:
c = mx.nd.concat(a, b, dim=1)

In [87]:
c = mx.nd.concat(c, b, dim=1)
c


[[ 1.  1.  1.  1.]
 [ 2.  2.  2.  2.]
 [ 3.  3.  3.  3.]
 [ 1.  2.  2.  2.]]
<NDArray 4x4 @cpu(0)>

In [88]:
c = mx.nd.concat(c, b, dim=1)
c


[[ 1.  1.  1.  1.  1.]
 [ 2.  2.  2.  2.  2.]
 [ 3.  3.  3.  3.  3.]
 [ 1.  2.  2.  2.  2.]]
<NDArray 4x5 @cpu(0)>